In [2]:
import pandas as pd
import sys
import os

# Add project root to path
sys.path.append(r"D:\data-problem-solving")

from db import load_table, engine

df_acc = pd.read_sql('SELECT * FROM "Premium vs Freemium".ms_acc_dimension', engine)
df_user = pd.read_sql('SELECT * FROM "Premium vs Freemium".ms_user_dimension', engine)
df_download = pd.read_sql('SELECT * FROM "Premium vs Freemium".ms_download_facts', engine)


In [4]:
# Join the three tables
cte = (
    df_acc.merge(df_user, on='acc_id')
          .merge(df_download, on='user_id')
)

# Group and aggregate
result = (
    cte.groupby('date')
       .agg(
           non_paying=('downloads', lambda x: x[cte.loc[x.index, 'paying_customer'] == 'no'].sum()),
           paying=('downloads', lambda x: x[cte.loc[x.index, 'paying_customer'] == 'yes'].sum())
       )
       .reset_index()
)

# Apply the HAVING condition
result = result[result['non_paying'] > result['paying']]

# Sort by date
result = result.sort_values('date')


In [5]:
print(result)

         date  non_paying  paying
1  2020-08-16          15      14
2  2020-08-17          45       9
3  2020-08-18          10       7
6  2020-08-21          32      17
